### remove value with >=99% missing


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD, PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline, make_union

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10000)

In [3]:
with open('train_iI26erW/train.csv', 'r') as fp, open('test_plBmD8c.csv') as fg:
    train = pd.read_csv(fp)
    test = pd.read_csv(fg)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(train.shape)
print(test.shape)

(300000, 333)
(200000, 332)


In [5]:
## count no. of NA per row

train['na_count'] = train.isnull().sum(axis=1)
test['na_count'] = test.isnull().sum(axis=1)

In [6]:
## net balance after spending in mon 1
addCols = [x for x in train.columns if 'CARD_' in x]
train['add_spend_mon1'] = train[addCols].sum(axis = 1) 
test['add_spend_mon1'] = test[addCols].sum(axis = 1)

train['add_spend_mon1'] = train['add_spend_mon1'].map(lambda x: x.count('Y'))
test['add_spend_mon1'] = test['add_spend_mon1'].map(lambda x: x.count('Y'))

In [7]:
train['CLOSED_DATE'] = train['CLOSED_DATE'].astype(str)
test['CLOSED_DATE'] = test['CLOSED_DATE'].astype(str)

In [8]:
def getDateFormat(val):
    if not val == 'nan':
        try:
            a = datetime.strptime(val, '%d%b%Y')
            b = a.strftime('%Y-%m-%d')
            return b
        except TypeError:
            return np.nan
    else:
        return np.nan

In [9]:
from datetime import datetime

train['closed_date_format'] = train['CLOSED_DATE'].map(getDateFormat)
test['closed_date_format'] = test['CLOSED_DATE'].map(getDateFormat)

train['maturity_gl_format'] = train['MATURITY_GL'].map(getDateFormat)
test['maturity_gl_format'] = test['MATURITY_GL'].map(getDateFormat)

train['maturity_lap_format'] = train['MATURITY_LAP'].map(getDateFormat)
test['maturity_lap_format'] = test['MATURITY_LAP'].map(getDateFormat)

train['maturity_las_format'] = train['MATURITY_LAS'].map(getDateFormat)
test['maturity_las_format'] = test['MATURITY_LAS'].map(getDateFormat)

In [10]:
## drop date formats
train.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)
test.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)

In [11]:
## date features

train['closed_date_format'] = pd.to_datetime(train['closed_date_format'])
test['closed_date_format'] = pd.to_datetime(test['closed_date_format'])

train['maturity_gl_format'] = pd.to_datetime(train['maturity_gl_format'])
test['maturity_gl_format'] = pd.to_datetime(test['maturity_gl_format'])

train['maturity_lap_format'] = pd.to_datetime(train['maturity_lap_format'])
test['maturity_lap_format'] = pd.to_datetime(test['maturity_lap_format'])

train['maturity_las_format'] = pd.to_datetime(train['maturity_las_format'])
test['maturity_las_format'] = pd.to_datetime(test['maturity_las_format'])

In [12]:
train['closed_date_weekday'] = train['closed_date_format'].dt.weekday
test['closed_date_weekday'] = test['closed_date_format'].dt.weekday

train['closed_date_year'] = train['closed_date_format'].dt.year
test['closed_date_year'] = test['closed_date_format'].dt.year

train['closed_date_month'] = train['closed_date_format'].dt.month
test['closed_date_month'] = test['closed_date_format'].dt.month

train['closed_date_day'] = train['closed_date_format'].dt.day
test['closed_date_day'] = test['closed_date_format'].dt.day

#########################################################

train['maturity_gl_weekday'] = train['maturity_gl_format'].dt.weekday
test['maturity_gl_weekday'] = test['maturity_gl_format'].dt.weekday

train['maturity_gl_year'] = train['maturity_gl_format'].dt.year
test['maturity_gl_year'] = test['maturity_gl_format'].dt.year

train['maturity_gl_month'] = train['maturity_gl_format'].dt.month
test['maturity_gl_month'] = test['maturity_gl_format'].dt.month

train['maturity_gl_day'] = train['maturity_gl_format'].dt.day
test['maturity_gl_day'] = test['maturity_gl_format'].dt.day

################################################################

train['maturity_lap_weekday'] = train['maturity_lap_format'].dt.weekday
test['maturity_lap_weekday'] = test['maturity_lap_format'].dt.weekday

train['maturity_lap_year'] = train['maturity_lap_format'].dt.year
test['maturity_lap_year'] = test['maturity_lap_format'].dt.year

train['maturity_lap_month'] = train['maturity_lap_format'].dt.month
test['maturity_lap_month'] = test['maturity_lap_format'].dt.month

train['maturity_lap_day'] = train['maturity_lap_format'].dt.day
test['maturity_lap_day'] = test['maturity_lap_format'].dt.day

######################################################################

train['maturity_las_weekday'] = train['maturity_las_format'].dt.weekday
test['maturity_las_weekday'] = test['maturity_las_format'].dt.weekday

train['maturity_las_year'] = train['maturity_las_format'].dt.year
test['maturity_las_year'] = test['maturity_las_format'].dt.year

train['maturity_las_month'] = train['maturity_las_format'].dt.month
test['maturity_las_month'] = test['maturity_las_format'].dt.month

train['maturity_las_day'] = train['maturity_las_format'].dt.day
test['maturity_las_day'] = test['maturity_las_format'].dt.day

######################################################################


In [13]:
formatCols = [x for x in train.columns if  '_format' in x]
train.drop(formatCols, axis=1, inplace=True)
test.drop(formatCols, axis=1, inplace=True)

In [14]:
train['net_emi_paid'] = train['HL_EMI'] / train['SCRUB_EMI']
test['net_emi_paid'] = test['HL_EMI'] / test['SCRUB_EMI']

train['total_salary_credited'] = train['SAL_MON_01'] + train['SAL_MON_02'] + train['SAL_MON_03']
test['total_salary_credited'] = test['SAL_MON_01'] + test['SAL_MON_02'] + test['SAL_MON_03']

In [15]:
train['total_balance'] = train['EOP_MON_02'] + train['EOP_MON_03'] + train['EOP_BAL_MON_01']
test['total_balance'] =  test['EOP_MON_02'] + test['EOP_MON_03'] + test['EOP_BAL_MON_01']

In [16]:
cardCols = [x for x in train.columns if 'CARD_' in x]

for x in cardCols:
    train[x] = train[x].map(lambda x: 1 if x == 'Y' else 0)
    test[x] = test[x].map(lambda x: 1 if x == 'Y' else 0)

In [17]:
train['spent_sum'] = train[cardCols].sum(axis=1)
test['spent_sum'] = test[cardCols].sum(axis=1)

In [18]:
dcount = [x for x in train.columns if 'D_COUNT' in x]
ccount = [x for x in train.columns if 'C_COUNT' in x]

train['debit_count'] = train[dcount].sum(axis=1)
train['credit_count'] = train[ccount].sum(axis=1)

test['debit_count'] = test[dcount].sum(axis=1)
test['credit_count'] = test[ccount].sum(axis=1)

In [19]:
damount = [x for x in train.columns if 'D_AMT' in x]
camount = [x for x in train.columns if 'C_AMT' in x]

train['debit_amount'] = train[damount].sum(axis=1)
train['credit_amount'] = train[camount].sum(axis=1)

test['debit_amount'] = test[damount].sum(axis=1)
test['credit_amount'] = test[camount].sum(axis=1)

In [20]:
train['DESIGNATION_FINAL'] = train['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)
test['DESIGNATION_FINAL'] = test['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)

In [21]:
dest_tr = pd.get_dummies(train['DESIGNATION_FINAL'])
det_te = pd.get_dummies(test['DESIGNATION_FINAL'])

In [22]:
train = pd.concat([train, dest_tr], axis=1)
test = pd.concat([test, det_te], axis=1)

In [23]:
train.drop('DESIGNATION_FINAL', axis=1, inplace=True)
test.drop('DESIGNATION_FINAL', axis=1, inplace=True)

In [24]:
ccspend = sorted([x for x in train.columns if 'CC_SPEND' in x])
dcspend = sorted([x for x in train.columns if 'DC_SPEND' in x])

In [25]:
train['credit_spent'] = train['CC_SPEND_MON_01'] + train['CC_SPEND_MON_02'] + train['CC_SPEND_MON_03'] +train['CC_SPEND_MON_04'] + train['CC_SPEND_MON_05'] + train['CC_SPEND_MON_06'] 
test['credit_spent'] = test['CC_SPEND_MON_01'] + test['CC_SPEND_MON_02'] + test['CC_SPEND_MON_03'] + test['CC_SPEND_MON_04'] + test['CC_SPEND_MON_05'] + test['CC_SPEND_MON_06'] 

train['debit_spent'] = train['DC_SPEND_MON_01'] + train['DC_SPEND_MON_02'] + train['DC_SPEND_MON_03'] +train['DC_SPEND_MON_04'] + train['DC_SPEND_MON_05'] + train['DC_SPEND_MON_06'] 
test['debit_spent'] = test['DC_SPEND_MON_01'] + test['DC_SPEND_MON_02'] + test['DC_SPEND_MON_03'] +test['DC_SPEND_MON_04'] + test['DC_SPEND_MON_05'] + test['DC_SPEND_MON_06'] 

In [26]:
cats= ['_ATM', '_CARE', '_ENT', '_HBY', '_HMD', '_HTL', '_JER', '_MED', '_RST', '_TRL']

In [27]:
## total sum by categories
for i in ccspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['cc_sum_'+j] = train[p].sum(axis=1)
        test['cc_sum_'+j] = test[p].sum(axis=1)

In [28]:
## total sum by debit category
for i in dcspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['dc_sum_'+j] = train[p].sum(axis=1)
        test['dc_sum_'+j] = test[p].sum(axis=1)

In [29]:
## total credit spent is greater than credit limit

train['is_exceed_credit'] = train.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis=1)
test['is_exceed_credit'] = test.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis =1)

In [30]:
## count live, closed loans
liveLoan = [x for x in train.columns if 'SCRUB_LIVE' in x]
closedLoan = [x for x in train.columns if 'SCRUB_CLOSED' in x]

train['live_loan_count'] = train[liveLoan].sum(axis=1)
train['closed_loan_count'] = train[closedLoan].sum(axis=1)

test['live_loan_count'] = test[liveLoan].sum(axis=1)
test['closed_loan_count'] = test[closedLoan].sum(axis=1)

In [31]:
txn_mon = [x for x in train.columns if 'TXN_MON' in x]
txn_mon = txn_mon[:-1]

In [32]:
for c in txn_mon:
    p = []
    if 'CC_TXN' in c:
        p.append(c)
    train['total_credit_tran'] = train[p].sum(axis=1)
    test['total_credit_tran'] = test[p].sum(axis=1)
    
for k in txn_mon:
    j = []
    if 'DC_TXN' in k:
        j.append(c)
    train['total_debit_tran'] = train[j].sum(axis=1)
    test['total_debit_tran'] = test[j].sum(axis=1)

In [33]:
print(train.shape)
print(test.shape)

(300000, 391)
(200000, 390)


### one hot encoding

In [34]:
objCols = train.select_dtypes(include=['object']).columns

In [35]:
countDict = {}
for x in objCols:
    if train[x].nunique() > 2:
        countDict[x] = train[x].nunique()
        
oneHotCols = list(countDict.keys())

In [36]:
## check if object columns in both datasets have same unique values

for x in oneHotCols:
    if train[x].nunique() != test[x].nunique():
        set_train = set(train[x].unique())
        set_test = set(test[x].unique())
        
        remove_train = set_train - set_test
        remove_test = set_test - set_train
        
        remove = remove_train.union(remove_test)
        
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        
        train[x] = train[x].map(filter_cat)
        test[x] = test[x].map(filter_cat)

In [37]:
oneHotCols = [x for x in oneHotCols if 'ZIP_CODE' not in x]

In [38]:
oneHot_train = pd.get_dummies(data=train[oneHotCols])
oneHot_test = pd.get_dummies(data=test[oneHotCols])

In [39]:
print(oneHot_train.shape)
print(oneHot_test.shape)

(300000, 221)
(200000, 221)


In [40]:
train = pd.concat([train, oneHot_train], axis=1)
test = pd.concat([test, oneHot_test], axis=1)

In [41]:
train.drop(oneHotCols, axis=1, inplace=True)
test.drop(oneHotCols, axis=1, inplace=True)

In [42]:
del oneHot_test, oneHot_train

In [43]:
import gc
gc.collect()

84

In [44]:
print(train.shape)
print(test.shape)

(300000, 603)
(200000, 602)


### label encoding

In [50]:
for x in train.columns:
    if x not in test.columns:
        print(x)

RESPONDERS


In [51]:
for x in test.columns:
    if x not in train.columns:
        print(x)

In [47]:
train.drop(['NEFT_CC_CATEGORY_WEDDING','NEFT_DC_CATEGORY_RETURN'], axis=1, inplace=True)
test.drop(['NEFT_CC_CATEGORY_CC_PAYMENT','NEFT_DC_CATEGORY_ADVANCE'], axis=1, inplace=True)

### remove columns with missing values - then label encode

In [52]:
misCols = train.isnull().sum(axis=0)/train.shape[0]
toremove = []

for k, v in misCols.iteritems():
    if v >= 0.99:
        toremove.append(k)

In [53]:
len(toremove)

111

In [55]:
train.drop('OCCUP_ALL_NEW', axis=1, inplace=True)
test.drop('OCCUP_ALL_NEW', axis=1, inplace=True)

In [56]:
train.drop(toremove, inplace=True, axis=1)
test.drop(toremove, inplace=True, axis=1)

In [57]:
for c in train.select_dtypes(include=['object']).columns:
    if c not in ['CUSTOMER_ID','RESPONDERS']:
        # print(c)
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [58]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [59]:
train['RESPONDERS'] = train['RESPONDERS'].replace(to_replace = {'Y':1, 'N':0})
target = train['RESPONDERS']

In [ ]:
### add pca features

### Do PCA and get new features

In [61]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [62]:
dim_train = train[feature_names]
dim_test = test[feature_names]

In [63]:
from sklearn.preprocessing import StandardScaler

In [64]:
rb = StandardScaler()

In [65]:
dim_train = rb.fit_transform(dim_train)
dim_test = rb.fit_transform(dim_test)

In [66]:
n_comp = 20

In [67]:
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_train = tsvd.fit_transform(dim_train)
tsvd_test = tsvd.transform(dim_test)

In [68]:
pca = PCA(n_components=n_comp, random_state=420)
pca_train = pca.fit_transform(dim_train)
pca_test = pca.transform(dim_test)

In [69]:
ica = FastICA(n_components=n_comp, random_state=2030)
ica_train = ica.fit_transform(dim_train)
ica_test = ica.transform(dim_test)

In [70]:
grp = GaussianRandomProjection(n_components=n_comp, random_state=42)
grp_train = grp.fit_transform(dim_train)
grp_test = grp.transform(dim_test)

In [71]:
srp = SparseRandomProjection(n_components=n_comp, random_state=42)
srp_train = srp.fit_transform(dim_train)
srp_test = srp.transform(dim_test)

In [72]:
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca_train[:,i-1]
    test['pca_' + str(i)] = pca_test[:,i-1]
    
    train['tsvd_' + str(i)] = tsvd_train[:,i-1]
    test['tsvd_' + str(i)] = tsvd_test[:,i-1]
    
    train['ica_' + str(i)] = ica_train[:,i-1]
    test['ica_' + str(i)] = ica_test[:,i-1]
    
    train['grp_' + str(i)] = grp_train[:,i-1]
    test['grp_' + str(i)] = grp_test[:,i-1]
    
    train['srp_' + str(i)] = srp_train[:,i-1]
    test['srp_' + str(i)] = srp_test[:,i-1]

In [73]:
del dim_train, dim_test

In [74]:
print(train.shape)
print(test.shape)

(300000, 589)
(200000, 588)


In [71]:
train.to_pickle('submissions/train_standard.pkl')
test.to_pickle('submissions/test_standard.pkl')

### CatBoost Model

In [76]:
trainX = train.drop(['CUSTOMER_ID','RESPONDERS'], axis=1)
testX = test.drop('CUSTOMER_ID', axis=1)

In [78]:
catCols = [trainX.columns.get_loc(k) for k in trainX.columns if trainX[k].dtype != np.float64]

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(trainX, target, train_size = 0.7, random_state = 42)

In [82]:
from catboost import CatBoostClassifier

In [83]:
model = CatBoostClassifier(custom_loss=['AUC'], ## changed AUC to cross entropy 
                           eval_metric = "AUC",
                           random_seed = 2017, 
                           learning_rate=0.1, 
                           iterations=1000, 
                           thread_count = 4,
                           rsm = 0.8,
                           depth = 5,
                           od_wait = 40,
                           od_type = 'Iter'
                          )

In [84]:
model.fit(X_train, y_train, cat_features=catCols, eval_set=(X_valid, y_valid), verbose=True, use_best_model=True)

Borders generated
feature 323 is redundant categorical feature, skipping it
feature 352 is redundant categorical feature, skipping it
feature 384 is redundant categorical feature, skipping it
feature 441 is redundant categorical feature, skipping it
feature 481 is redundant categorical feature, skipping it
0:	learn 0.6863957151	test 0.682273962	bestTest 0.682273962	passed: 18.4 sec	total: 2m 56s	remaining: 2d 56m 53s
1:	learn 0.6863957151	test 0.682273962	bestTest 0.682273962	passed: 1.7 sec	total: 2m 58s	remaining: 1d 41m 9s
2:	learn 0.6863843752	test 0.682273962	bestTest 0.682273962	passed: 15.8 sec	total: 3m 13s	remaining: 17h 53m 50s
3:	learn 0.6864167713	test 0.6822976406	bestTest 0.6822976406	passed: 21 sec	total: 3m 34s	remaining: 14h 51m 45s
4:	learn 0.7079066362	test 0.7055840979	bestTest 0.7055840979	passed: 7.61 sec	total: 3m 42s	remaining: 12h 17m 56s
5:	learn 0.7079066362	test 0.7055840979	bestTest 0.7055840979	passed: 1.31 sec	total: 3m 43s	remaining: 10h 17m 56s
6:	learn

KeyboardInterrupt: 

In [ ]:
preds1 = model.predict_proba(testX)[:,1]